# Homework 1

**Course**: FFR120 - Simulation of Complex Systems

**Author**: Artur Gasparyan

**Assignment**: Homework 1 - Option 4

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim

In [ ]:
rng = np.random.default_rng()
save_file = False
framerate_fast = 10
framerate_medium = 5
framerate_slow = 2
gif_writer_fast = anim.PillowWriter(fps=framerate_fast)
gif_writer_medium = anim.PillowWriter(fps=framerate_medium)
gif_writer_slow = anim.PillowWriter(fps=framerate_slow)

## Exercise 4.1 

In [ ]:
import math

def decimal_to_binary_array(dec, width):
    if width is None:
        if dec == 0:
            width = 1
        else:
            width = 1 + math.floor(math.log2(dec))
    
    array = np.zeros(shape=(width,), dtype='uint8')
    
    for i in range(width-1, -1, -1):
        dec, array[i] = divmod(dec, 2)

    return array


In [ ]:
def create_parent_generation(n_cells, rule_nr, rng):
    if rule_nr in (90, 30):
        parent_generation = np.zeros((n_cells,))
        parent_generation[n_cells//2] = 1
        
    else:
        parent_generation = rng.integers(0, 2, size=(n_cells,))
        
    return parent_generation


def create_grid_1d(rule_nr, n_cells, n_generations, rng):
    grid = np.zeros(shape=(n_generations, n_cells), dtype='uint8')
    grid[0, :] = create_parent_generation(n_cells, rule_nr, rng) 
    return grid


def decode_1d_rule(rule_number):
    binary_numbers = [
        (1, 1, 1), (1, 1, 0), (1, 0, 1), (1, 0, 0),
        (0, 1, 1), (0, 1, 0), (0, 0, 1), (0, 0, 0)
    ]

    rule = dict()
    num = rule_number
    for i in range(8):
        num, rem = divmod(num, 2)
        rule[binary_numbers[-i-1]] = rem

    return rule


def update_grid_1d(grid, n_cells, rules, rng):
    grid = np.roll(grid, 1, 0)
    for i in range(1, n_cells-1):
        pattern = grid[1, i-1:i+2]
        grid[0, i] = rules[tuple(pattern)]

    left_pattern = np.zeros(shape=(3,))
    left_pattern[0] = grid[1, -1]
    left_pattern[1:] = grid[1, :2]

    right_pattern = np.zeros(shape=(3,))
    right_pattern[-1] = grid[1, 0]
    right_pattern[:2] = grid[1, -2:]

    grid[0, 0] = rules[tuple(left_pattern)]
    grid[0, -1] = rules[tuple(right_pattern)]

    return grid


In [ ]:
def run_simulation_1d(n_cells, n_generations, rule_nr, rng):
    rules = decode_1d_rule(rule_number=rule_nr)
    grid = create_grid_1d(rule_nr, n_cells, n_generations, rng)
    history = []

    for gen in range(n_generations):
        grid = update_grid_1d(grid, n_cells, rules, rng)
        history.append(grid)

    return history


In [ ]:
def render_grid(time, history, img):
    img.set_array(history[time])
    return img,


def create_animation(history, framerate):
    fig = plt.figure()
    img = plt.imshow(history[0], interpolation='none', animated=True)
    animation = anim.FuncAnimation(fig=fig, func=render_grid, frames=len(history), interval=1000/framerate, fargs=(history,img), blit=True)    
    return animation


def animate_simulation(name, history, framerate, writer, save_file):
    animation = create_animation(history, framerate)
    if save_file:
        animation.save(f"./gifs/{name}.gif", writer=writer)
        
    return animation
    

In [ ]:
n_cells = 80
n_generations = 80

In [ ]:
history_184 = run_simulation_1d(n_cells, n_generations, 184, rng)
animation_184 = animate_simulation("evolution-184", history_184, framerate_fast, gif_writer_fast, save_file)

In [ ]:
history_90 = run_simulation_1d(n_cells, n_generations, 90, rng)
animation_90 = animate_simulation("evolution-90", history_90, framerate_fast, gif_writer_fast, save_file)

In [ ]:
history_30 = run_simulation_1d(n_cells, n_generations, 30, rng)
animation_30 = animate_simulation("evolution-30", history_30, framerate_fast, gif_writer_fast, save_file)

In [ ]:
history_110 = run_simulation_1d(n_cells, n_generations, 110, rng)
animation_110 = animate_simulation("evolution-110", history_110, framerate_fast, gif_writer_fast, save_file)

In [ ]:
fig, axes = plt.subplots(2, 2)
histories = [history_184, history_90, history_30, history_110]
plt_names = ["Rule 184", "Rule 90", "Rule 30", "Rule 110"]
fig.tight_layout(pad=2)

for hist, ax, name in zip(histories, axes.flatten(), plt_names):
    ax.imshow(hist[-1])
    ax.set_title(name)
    

## Exercise 4.2

In [ ]:
def create_grid(grid_size, rng):
    return rng.integers(0, 2, size=grid_size)
    

def enough_nbs_to_live(value, n_nbs):
    return n_nbs == 3 or (n_nbs == 2 and value)


def count_nbs(grid, grid_size, is_periodic_boundary):
    h, w = grid_size
    counts = np.zeros(shape=grid_size, dtype='uint8')
    
    if is_periodic_boundary:
        for y in range(h):
            for x in range(w):
                n_nbs = 0
                for nb_y in range(y-1, y+2):
                    nb_ym = nb_y % h
                    for nb_x in range(x-1, x+2):
                        nb_xm = nb_x % w
                        n_nbs += grid[nb_ym, nb_xm]
                        
                counts[y, x] = n_nbs - grid[y, x]
    
    else:
        for y in range(h):
            for x in range(w):
                n_nbs = 0
                for nb_y in range(max(y-1, 0), min(y+2, h-1)):
                    for nb_x in range(max(x-1, 0), min(x+2, w-1)):
                        n_nbs += grid[nb_y, nb_x]

                counts[y, x] = n_nbs - grid[y, x]
            
    return counts
            
            
def calculate_new_gen(grid, counts):
    return ((counts==3) | ((counts==2) & (grid==1))).astype('uint8')


def update_grid(grid, grid_size, is_periodic_boundary):
    counts = count_nbs(grid, grid_size, is_periodic_boundary)
    new_gen = calculate_new_gen(grid, counts)
    return new_gen


def run_simulation(initial_grid, n_generations, is_periodic_boundary):
    grid_size = initial_grid.shape
    grid = initial_grid
    history = [grid]
    
    for gen in range(1, n_generations):
        grid = update_grid(grid, grid_size, is_periodic_boundary)
        history.append(grid)
        
    return history

In [ ]:
grid_size = 10, 10
n_generations = 20
initial_grid = create_grid(grid_size, rng)

In [ ]:
history_nonperiodic = run_simulation(initial_grid, n_generations, is_periodic_boundary=False)
animation_nonperiodic = animate_simulation(
    "evolution-nonperiodic", history_nonperiodic, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
history_periodic = run_simulation(initial_grid, n_generations, is_periodic_boundary=True)
animation_periodic = animate_simulation(
    "evolution-periodic", history_periodic, framerate_medium, gif_writer_medium, save_file
)

## Exercise 4.3

In [ ]:
def simulate_block():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 2, False)

history_block = simulate_block()
animation_block = animate_simulation(
    "animation_block", history_block, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_beehive():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 2, False)

history_beehive = simulate_beehive()
animation_beehive = animate_simulation(
    "animation_beehive", history_beehive, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_loaf():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 2, False)
    
history_loaf = simulate_loaf()
animation_loaf = animate_simulation(
    "animation_loaf", history_loaf, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_boat():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 2, False)
    
history_boat = simulate_boat()
animation_boat = animate_simulation(
    "animation_boat", history_boat, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_tub():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 2, False)
    
history_tub = simulate_tub()
animation_tub = animate_simulation(
    "animation_tub", history_tub, framerate_medium, gif_writer_medium, save_file
)

## Exercise 4.4

In [ ]:
def simulate_blinker():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 4, False)
    
history_blinker = simulate_blinker()
animation_blinker = animate_simulation(
    "animation_blinker", history_blinker, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_toad():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 4, False)
    
history_toad = simulate_toad()
animation_toad = animate_simulation(
    "animation_toad", history_toad, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_beacon():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 4, False)
    
history_beacon = simulate_beacon()
animation_beacon = animate_simulation(
    "animation_beacon", history_beacon, framerate_medium, gif_writer_medium, save_file
)

## Exercise 4.5

In [ ]:
def simulate_glider_1():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 24, True)
    
history_glider_1 = simulate_glider_1()
animation_glider_1 = animate_simulation(
    "animation_glider_1", history_glider_1, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_glider_2():
    grid = np.array([
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 24, True)
    
history_glider_2 = simulate_glider_2()
animation_glider_2 = animate_simulation(
    "animation_glider_2", history_glider_2, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_glider_3():
    grid = np.array([
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 24, True)
    
history_glider_3 = simulate_glider_3()
animation_glider_3 = animate_simulation(
    "animation_glider_3", history_glider_3, framerate_medium, gif_writer_medium, save_file
)

In [ ]:
def simulate_glider_4():
    grid = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
    ], dtype='uint8')
    
    return run_simulation(grid, 24, True)
    
history_glider_4 = simulate_glider_4()
animation_glider_4 = animate_simulation(
    "animation_glider_4", history_glider_4, framerate_medium, gif_writer_medium, save_file
)

## Exercise 4.6

In [ ]:
def find_pattern_in_grid(rng, config_size, max_gen):
    config = generate_config(rng, config_size)
    if np.sum(config) == 0:
        raise ValueError("config has all zeros: ", config)

    grid_size = (3*config_size[0], 3*config_size[1]) 
    grid = np.zeros(shape=grid_size, dtype='uint8')
    start_pos = config_size
    grid[start_pos[0]:start_pos[0]+config_size[0],
         start_pos[1]:start_pos[1]+config_size[1]] = config
    
    initial_grid = grid
    

    for gen in range(1, max_gen):
        grid = update_grid(grid, grid_size, False)
        pos = find_config_shift(grid, config, config_size)
        if pos:
            shift = (pos[0]-start_pos[0], pos[1]-start_pos[1])
            return gen, shift, config, initial_grid, grid
    
    return None


def shift_config(grid, config_pos, config_size, shift):
    y, x = config_pos
    h, w = config_size
    dy, dx = shift
    config = grid[y:y+h, x:x+w]

    grid[y:y+h, x:x+w] = 0
    grid[y+dy:y+dy+h, x+dx:x+dx+w] = config

    return grid


def find_config_shift(grid, config, config_size): 
    h, w = config_size

    for y in range(0, 2*h):
        for x in range(0, 2*w):
            if np.array_equal(grid[y:y+h, x:x+w], config):
                return y, x

    return None


In [ ]:
i_still = 0
i_oscillator = 1
i_glider = 2

def find_patterns(rng, n_tries, config_size, max_gen):
    found_patterns = [[], [], []]
    found_identifiers = set()
    
    n_counts = [0, 0, 0]
    n_unique = [0, 0, 0]

    print("Looking for patterns...")
    for t in range(n_tries):
        if t%100 == 0:
            print(f"{t}/{n_tries}, counts={n_counts}, unique={n_unique}")

        found_info = find_pattern_in_grid(rng, config_size, max_gen)
        
        if found_info:
            gen, shift, *_ = found_info
            
            if shift != (0, 0):
                index = i_glider
            elif gen == 1:
                index = i_still
            else:
                index = i_oscillator
                
            n_counts[index] += 1
            
            identifier = (gen, shift)
            if identifier in found_identifiers:
                continue
            else:
                found_identifiers.add(identifier)

            n_unique[index] += 1
            found_patterns[index].append(found_info)

    if len(found_patterns) == 0:
        print("No pattern found")
    else:
        print(f"Finished! Found {len(found_patterns[i_still])} still-lifes, "
          f"{len(found_patterns[i_oscillator])} oscillators, and "
          f"{len(found_patterns[i_glider])} gliders.")
        
        print()
        print(f"Probability of still life: {n_counts[0] / n_tries}")
        print(f"Probability of oscillator: {n_counts[1] / n_tries}")
        print(f"Probability of glider: {n_counts[2] / n_tries}")
        print()
        print(f"Probability of unique: {sum(n_unique) / n_tries}")
        print(f"Probability of duplicate: {(sum(n_counts)-sum(n_unique))/n_tries}")
       
    return found_patterns, n_counts, n_unique


In [ ]:
n_tries = 1000
config_size = 5, 5
max_gen = 8

found_patterns, n_counts, n_unique = find_patterns(rng, n_tries, config_size, max_gen)

In [ ]:
def plot_found_patterns(found_patterns, max_of_each=None):
    mixed_patterns = (
        found_patterns[i_still][:max_of_each] +
        found_patterns[i_oscillator][:max_of_each] +
        found_patterns[i_glider][:max_of_each]
    )

    fig, axes = plt.subplots(len(mixed_patterns), 2, figsize=(7, 17))
    fig.tight_layout()
    
    for i, (gen, shift, config, initial_grid, grid) in enumerate(mixed_patterns):    
        ax1, ax2 = axes[i, :]
        ax1.imshow(initial_grid)
        ax1.set_title("Initial (dx=0, dy=0, gen=0)")
        ax2.imshow(grid)
        ax2.set_title(f"Shifted (dx={shift[1]}, dy={shift[0]}, gen={gen})")


In [ ]:
plot_found_patterns(found_patterns)

The function is quite ineffecient for finding new patterns since it samples randomly, which results in many duplicates.
A better way could be to only use unique seeds. Create a list of random and unique seeds and use every single one from it.